# Experiments Notebook

This notebook is for running and documenting model experiments.

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

from src.models.unet_smp import create_model
from src.models.losses import get_loss_function
from src.training.dataset import BuildingDataset
from src.utils.metrics import compute_metrics
from src.utils.vis import plot_prediction
from src.utils.config import load_config

%matplotlib inline

## 1. Setup

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

if device == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## 2. Load Configuration

In [ ]:
config_path = project_root / 'configs' / 'train.yaml'

if config_path.exists():
    config = load_config(str(config_path))
    print('Configuration loaded')
else:
    print('Config file not found')
    config = {}

## 3. Create Model

In [ ]:
model_config = config.get('model', {})

model = create_model(
    name=model_config.get('name', 'unet'),
    encoder=model_config.get('encoder', 'resnet34'),
    encoder_weights=model_config.get('encoder_weights', 'imagenet'),
    in_channels=model_config.get('in_channels', 3),
    classes=model_config.get('classes', 1),
)

model = model.to(device)
print(f'Model created: {model_config.get("name", "unet")} with {model_config.get("encoder", "resnet34")} encoder')

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total parameters: {total_params:,}')
print(f'Trainable parameters: {trainable_params:,}')

## 4. Test Forward Pass

In [ ]:
x = torch.randn(4, 3, 512, 512).to(device)

model.eval()
with torch.no_grad():
    y = model(x)

print(f'Input shape: {x.shape}')
print(f'Output shape: {y.shape}')

## 5. Load Dataset (if available)

In [ ]:
data_config = config.get('data', {})
tiles_dir = project_root / data_config.get('tiles_dir', 'data/processed/tiles')

train_images_dir = tiles_dir / 'train' / 'images'
train_masks_dir = tiles_dir / 'train' / 'masks'

if train_images_dir.exists():
    transform = A.Compose([
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    
    train_dataset = BuildingDataset(
        images_dir=str(train_images_dir),
        masks_dir=str(train_masks_dir),
        transform=transform,
    )
    
    print(f'Train dataset size: {len(train_dataset)}')
else:
    print('Training data not found. Run preprocessing first.')

## 6. Test Loss Functions

In [ ]:
loss_configs = [
    {'name': 'bce'},
    {'name': 'dice'},
    {'name': 'dice_bce', 'bce_weight': 0.5, 'dice_weight': 0.5},
    {'name': 'focal', 'alpha': 0.25, 'gamma': 2.0},
    {'name': 'tversky', 'alpha': 0.5, 'beta': 0.5},
]

pred = torch.randn(4, 1, 256, 256).to(device)
target = torch.randint(0, 2, (4, 1, 256, 256)).float().to(device)

print('Loss function comparison:')
for loss_config in loss_configs:
    criterion = get_loss_function(loss_config)
    loss = criterion(pred, target)
    print(f'{loss_config["name"]}: {loss.item():.4f}')

## 7. Compare Encoders

In [ ]:
encoders = ['resnet18', 'resnet34', 'resnet50', 'efficientnet-b0', 'efficientnet-b2']

print('Encoder comparison:')
print('-' * 60)

for encoder in encoders:
    try:
        model = create_model(
            name='unet',
            encoder=encoder,
            encoder_weights=None,
            in_channels=3,
            classes=1,
        )
        
        params = sum(p.numel() for p in model.parameters())
        
        model = model.to(device)
        model.eval()
        
        x = torch.randn(1, 3, 512, 512).to(device)
        
        import time
        torch.cuda.synchronize() if device == 'cuda' else None
        start = time.time()
        
        with torch.no_grad():
            for _ in range(10):
                _ = model(x)
        
        torch.cuda.synchronize() if device == 'cuda' else None
        elapsed = (time.time() - start) / 10
        
        print(f'{encoder:20s} | Params: {params/1e6:6.2f}M | Time: {elapsed*1000:.1f}ms')
        
        del model
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f'{encoder:20s} | Error: {e}')

## 8. Load Checkpoint and Evaluate (if available)

In [ ]:
checkpoint_path = project_root / 'checkpoints' / 'best.pth'

if checkpoint_path.exists():
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    print('Checkpoint info:')
    print(f'Epoch: {checkpoint.get("epoch", "N/A")}')
    print(f'Metrics: {checkpoint.get("metrics", {})}')
else:
    print('No checkpoint found. Train the model first.')

## 9. Experiment Notes

Document your experiment findings here:

- Experiment 1: ...
- Experiment 2: ...
- Observations: ...